In [13]:
from PIL import Image
import cupy as cp
import numpy as np

In [14]:
# Load the four band images
band1 = Image.open('band1.gif')
band2 = Image.open('band2.gif')
band3 = Image.open('band3.gif')
band4 = Image.open('band4.gif')

# Convert the images to numpy arrays
band1 = np.array(band1)
band2 = np.array(band2)
band3 = np.array(band3)
band4 = np.array(band4)

# Stack the band images along the third axis
data = np.stack((band1, band2, band3, band4), axis=-1)

# Reshape the data into a two-dimensional array
data = data.reshape(-1, 4)

In [16]:
# Split the data into training, validation, and testing sets
np.random.seed(42)
np.random.shuffle(data)
train_size = int(0.6 * data.shape[0])
val_size = int(0.2 * data.shape[0])
X_train = cp.array(data[:train_size, :])
X_val = cp.array(data[train_size:train_size+val_size, :])
X_test = cp.array(data[train_size+val_size:, :])


In [22]:
# Define the k-means function for GPU acceleration
def k_means(X, k, max_iterations=100):
    # Initialize the centroids randomly
    indices = cp.random.choice(X.shape[0], k, replace=False)
    centroids = X[indices, :]

    # Iterate until convergence or max_iterations
    for i in range(max_iterations):
        # Assign each point to the closest centroid
        distances = cp.linalg.norm(X[:, cp.newaxis, :] - centroids[cp.newaxis, :, :], axis=-1)
        labels = cp.argmin(distances, axis=1)

        # Update the centroids to the mean of the assigned points
        for j in range(k):
            mask = labels == j
            if cp.sum(mask) > 0:
                centroids[j, :] = cp.mean(X[mask, :], axis=0)

    return centroids, labels

In [23]:
# Run k-means for k = 2, 3, 4, 5, 6, 7 on the training set
for k in range(2, 8):
    centroids, labels = k_means(X_train, k)
    
    # Plot the cluster assignments as an image with proper colors
    clusters = cp.zeros(labels.shape + (3,), dtype=cp.uint8)
    for i in range(k):
        clusters[labels == i, :] = cp.array(Image.open(f"band{i+1}.gif"))
    clusters = clusters.reshape(band1.shape[0], band1.shape[1], 3)
    Image.fromarray(clusters).show()

ValueError: operands could not be broadcast together with shape (512, 512) and requested shape (58747, 3)

In [24]:
import cupy as cp
from PIL import Image

# Load band images
band1 = cp.array(Image.open("band1.gif"))
band2 = cp.array(Image.open("band2.gif"))
band3 = cp.array(Image.open("band3.gif"))
band4 = cp.array(Image.open("band4.gif"))

# Reshape the data to 2D array
data = cp.vstack((band1.ravel(), band2.ravel(), band3.ravel(), band4.ravel())).T

# Number of clusters
k = 2

# Initialize centroids randomly
centroids = cp.random.rand(k, 4)

# Iterate until convergence
for _ in range(10):
    # Assign labels to each datapoint based on centroids
    distances = cp.sqrt(((data - centroids[:, np.newaxis])**2).sum(axis=2))
    labels = cp.argmin(distances, axis=0)
    
    # Update centroids based on the labels
    for i in range(k):
        centroids[i] = data[labels == i].mean(axis=0)
        
# Reshape the labels to have the same shape as the band arrays
labels = labels.reshape(band1.shape)

# Create color image from labels
clusters = cp.zeros(labels.shape + (3,), dtype=cp.uint8)
for i in range(k):
    cluster_color = cp.random.rand(3) * 255
    clusters[labels == i, :] = cp.array(cluster_color).astype(cp.uint8)

# Display the image
Image.fromarray(clusters).show()


AttributeError: 'cupy.core.core.ndarray' object has no attribute '__array_interface__'